# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
#!pip install pandas

In [3]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [72]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld,I loved this product. But they only seem to l...


## LLMChain

In [7]:
!pip install langchain --upgrade

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 9.6 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.0
    Uninstalling langchain-0.3.0:
      Successfully uninstalled langchain-0.3.0


In [9]:
pip install langchain_openai

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ---------------------------------------- 0.0/884.2 kB ? eta -:--:--
   ---------------------------------------- 884.2/884.2 kB 9.9 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [11]:
#Replace None by your own value and justify

# Temperature set to 0.7 to balance creativity and coherence in the responses.
# Lower values make outputs more deterministic, while higher values increase randomness.

llm = ChatOpenAI(temperature=0.7)


In [12]:
prompt = ChatPromptTemplate.from_template( "Describe the following product: {product}" #Write a query that would take a variable to describe any product
 
)

In [13]:

chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\larry\AppData\Local\Temp\ipykernel_7528\546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


# **Replace the use of LLMChain:**
from langchain.schema import RunnableSequence

*# Create the RunnableSequence*
chain = RunnableSequence.from_runnables([prompt, llm])

*# Run the chain*
product = "Noise-Cancelling Wireless Headphones"
response = chain.invoke({"product": product})
print(response)

In [15]:
product = "Noise-Cancelling Wireless Headphones" #Select a product type to be describe
chain.run(product)

C:\Users\larry\AppData\Local\Temp\ipykernel_7528\1589426625.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'Noise-cancelling wireless headphones are a type of headphones that use advanced technology to reduce or eliminate external noise, allowing the user to enjoy their music or audio content without distractions. These headphones are designed to block out ambient noise such as traffic, conversations, or airplane engine noise, creating a more immersive listening experience. Additionally, they are wireless, meaning they connect to devices via Bluetooth, providing freedom of movement without the hassle of tangled cords. This product is ideal for individuals who want to listen to music or podcasts in noisy environments or during travel.'

## SimpleSequentialChain

In [16]:
from langchain.chains import SimpleSequentialChain

In [17]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Describe the following product: {product}" #Repeat the initial query or create a new query that would feed into the second prompt
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [26]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    """Here is the description: {output}. 
    What are the key benefits of this product?""" #Write the second prompt query that takes an input variable whose input will come from the previous prompt"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [27]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [28]:
product = "Noise-Cancelling Wireless Headphones"

In [29]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Noise-cancelling wireless headphones are a type of headphones that use technology to reduce or eliminate external background noise, allowing the user to listen to music or other audio content without distraction. They connect to devices wirelessly via Bluetooth, providing freedom of movement without the hassle of tangled cords. The headphones typically have built-in controls for adjusting volume and play/pause functions. They are designed to provide a high-quality audio experience with clear sound and deep bass. The noise-cancelling feature works by using microphones to pick up ambient noise and producing sound waves that cancel out the unwanted noise. This makes them ideal for use in loud environments such as airplanes, trains, or busy offices.
1. Improved audio experience: Noise-cancelling wireless headphones allow the user to enjoy their music or audio content without being disturbed by external noise, providing a clear and immersive l

'1. Improved audio experience: Noise-cancelling wireless headphones allow the user to enjoy their music or audio content without being disturbed by external noise, providing a clear and immersive listening experience.\n\n2. Wireless convenience: With Bluetooth connectivity, these headphones offer freedom of movement without the restrictions of tangled cords, making them ideal for use while on the go or during physical activities.\n\n3. Built-in controls: The headphones typically come with controls for adjusting volume, play/pause functions, and skipping tracks, allowing for easy and convenient operation without needing to access the connected device.\n\n4. Noise-cancelling technology: The active noise-cancelling feature helps to reduce or eliminate external background noise, making them ideal for use in noisy environments such as airplanes, trains, or busy offices, allowing the user to focus on their audio content.\n\n5. High-quality sound: Noise-cancelling wireless headphones are desi

**Repeat the above twice for different products**

In [31]:
# Second product: Smart Home Assistant
product_2 = "Smart Home Assistant"
overall_simple_chain.run(product_2)



> Entering new SimpleSequentialChain chain...
A smart home assistant is a device that connects to your home's Wi-Fi network and uses artificial intelligence to assist with various tasks around the house. It typically comes in the form of a speaker that can respond to voice commands and perform functions such as playing music, setting alarms and timers, answering questions, controlling smart home devices like lights and thermostats, and providing updates on the weather and news. Some popular examples of smart home assistants include Amazon Echo, Google Home, and Apple HomePod. These devices can also be integrated with other smart home products to create a seamless and convenient home automation system.
Some key benefits of smart home assistants include:

1. Convenience: Smart home assistants can help streamline daily tasks and make it easier to control various aspects of your home with just your voice, saving you time and effort.

2. Hands-free operation: With the ability to respond t

'Some key benefits of smart home assistants include:\n\n1. Convenience: Smart home assistants can help streamline daily tasks and make it easier to control various aspects of your home with just your voice, saving you time and effort.\n\n2. Hands-free operation: With the ability to respond to voice commands, smart home assistants allow you to access information and control devices without having to physically interact with them.\n\n3. Home automation: By integrating with other smart home devices, you can create a cohesive home automation system that allows you to control multiple aspects of your home with your smart home assistant.\n\n4. Personalized experience: Smart home assistants can learn your preferences over time and provide personalized recommendations and information based on your habits and interests.\n\n5. Entertainment: Smart home assistants can play music, podcasts, audiobooks, and radio stations, providing entertainment options for you and your family.\n\n6. Information a

In [32]:
# Third product: Wireless Charging Pad
product_3 = "Wireless Charging Pad"
result_3 = overall_simple_chain.run(product_3)
print("\nResults for Wireless Charging Pad:\n", result_3)



> Entering new SimpleSequentialChain chain...
A wireless charging pad is a device used to charge electronic devices such as smartphones, tablets, and smartwatches without the need for cables or cords. It works by utilizing electromagnetic induction to transfer power from the pad to the device. The device is simply placed on the pad, and it starts charging automatically. Wireless charging pads are convenient, clutter-free, and efficient, making them a popular choice for individuals looking for a hassle-free way to charge their devices.
1. Convenience: Wireless charging pads offer a convenient way to charge electronic devices without the need for cables or cords. Users simply need to place their device on the pad, and it will start charging automatically.

2. Clutter-free: With no cables or cords to deal with, wireless charging pads help reduce clutter on desks, nightstands, and other surfaces. This can create a more organized and aesthetically pleasing environment.

3. Efficiency: Wir

## SequentialChain

In [33]:
from langchain.chains import SequentialChain

In [54]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template( "Translate the following review into English: {review}"
  #This prompt should translate a review
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="translated_review" #Give a name to your output
                    )

In [55]:
second_prompt = ChatPromptTemplate.from_template("Summarize the following review: {translated_review}"
    #Write a prompt to summarize a review
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary" #give a name to this output
                    )


In [56]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template( "Analyze the following review summary and determine its tone (positive, negative, or neutral): {summary}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="tone"
                      )

In [57]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template( """Based on the tone '{tone}' and the summary '{summary}', recommend an action to the user.
    If the tone is positive, suggest buying the product. If the tone is negative, suggest not buying it.
    If the tone is neutral, suggest evaluating other products."""
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="recommendation"
                     )


In [58]:
# overall_chain: input= Review 
# and output= English Review,summary, tone & recommendation
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["translated_review", "summary", "tone", "recommendation"],
    verbose=True
)

In [67]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'translated_review': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer finds the taste of the product to be mediocre and is disappointed that the foam does not hold well. They suspect that the product may be an old batch or counterfeit, as they have had better experiences with the same product purchased in stores.',
 'tone': 'Negative',
 'recommendation': 'Do not buy the product as the reviewer had a negative experience with it. It is recommended to find a different option that may better suit your needs.'}

**Repeat the above twice for different products or reviews**

In [68]:
review = df.Review[6]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': 'Está lu bonita calienta muy rápido, es muy funcional, solo falta ver cuánto dura, solo llevo 3 días en funcionamiento.',
 'translated_review': 'This heater is very beautiful and heats up very quickly, it is very functional, I just need to see how long it lasts, I have only been using it for 3 days.',
 'summary': 'The reviewer finds the heater to be beautiful and functional, heating up quickly. They are unsure of its longevity as they have only been using it for three days.',
 'tone': 'The tone of the review summary is generally positive. The reviewer praises the heater for being beautiful and functional, and also notes that it heats up quickly. However, there is a hint of uncertainty regarding the longevity of the heater since they have only been using it for a short period of time.',
 'recommendation': "Since the tone of the review summary is generally positive, I would recommend buying the heater. The reviewer praises its beauty, functionality, and quick heating, which ar

In [73]:
review = df.Review[7]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': 'Funciona segun dice la promocion. No es el mejor que haya utilizado. No calienta tanto como otras marcas. No lo devolvere, pero no necesariamente compraria otro. Hay otras marcas mejores pero este resuelve. Para el precio esta ok. Pero si fuera mas caro no lo compraria.',
 'translated_review': "It works as advertised. It's not the best I've used. It doesn't heat up as much as other brands. I won't return it, but I wouldn't necessarily buy another one. There are better brands out there, but this one does the job. For the price, it's okay. But if it were more expensive, I wouldn't buy it.",
 'summary': "The reviewer found that the product functions as advertised, but they have used better brands in the past. They noted that the product doesn't heat up as much as other brands and while they won't return it, they wouldn't necessarily buy it again. The reviewer believes there are better brands available, but for the price, it is adequate. If the product were more expensive, they

## Router Chain

In [133]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""


history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""


biology_template = """You are a very knowledgeable biologist. \
You are great at answering questions about biology in a clear and concise manner. \
You make complex biological concepts easy to understand. \
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{input}"""

In [134]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering questions about biology",
        "prompt_template": biology_template
    }
]

In [135]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [136]:
llm = ChatOpenAI(temperature=0)

In [137]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [138]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [139]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [140]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [141]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [142]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases."

In [143]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'2 + 2 is equal to 4.'

In [145]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA carries the genetic information that determines the characteristics of an organism. This genetic information is essential for the cell to function properly and carry out its specific roles within the body. DNA contains the instructions for making proteins, which are essential for the structure, function, and regulation of cells. Additionally, DNA is passed on from parent to offspring, ensuring that genetic information is inherited and maintained throughout generations.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [146]:
chain.run("Why does the sky have a blue color?")



> Entering new MultiPromptChain chain...
physics: {'input': 'Why does the sky have a blue color?'}
> Finished chain.


"The sky appears blue because of a phenomenon called Rayleigh scattering. When sunlight enters Earth's atmosphere, it interacts with the gases and particles in the air. The shorter blue wavelengths of light are scattered in all directions by the gases and particles, while the longer wavelengths of light (like red and yellow) pass straight through. This scattering of blue light is what gives the sky its blue color."

In [147]:
chain.run("What do I need to create a simple diy solar system for about 5kWh?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What do I need to create a simple diy solar system for about 5kWh?'}
> Finished chain.


'To create a simple DIY solar system that can generate about 5kWh of energy, you will need the following components:\n\n1. Solar panels: You will need enough solar panels to generate 5kWh of energy per day. The number of panels will depend on their efficiency and the amount of sunlight your location receives.\n\n2. Inverter: You will need an inverter to convert the DC electricity generated by the solar panels into AC electricity that can be used in your home.\n\n3. Charge controller: A charge controller is necessary to regulate the flow of electricity from the solar panels to the batteries, preventing overcharging and extending the lifespan of the batteries.\n\n4. Batteries: You will need batteries to store the excess electricity generated by the solar panels for use when the sun is not shining.\n\n5. Mounting hardware: You will need mounting hardware to securely install the solar panels on your roof or in your yard.\n\n6. Wiring and connectors: You will need wiring and connectors to c

In [148]:
chain.run("Which is the best virtual immersive game available?")



> Entering new MultiPromptChain chain...
None: {'input': 'Which is the best virtual immersive game available?'}
> Finished chain.


'There are many virtual immersive games available, and the best one can vary depending on personal preferences. Some popular virtual immersive games include:\n\n1. "Half-Life: Alyx" - A highly immersive virtual reality game set in the Half-Life universe, with stunning graphics and engaging gameplay.\n\n2. "Beat Saber" - A rhythm-based virtual reality game where players use lightsabers to slice through blocks to the beat of the music.\n\n3. "The Elder Scrolls V: Skyrim VR" - A virtual reality version of the popular open-world RPG game, allowing players to explore the world of Skyrim in a whole new way.\n\n4. "Superhot VR" - A unique virtual reality game where time only moves when you move, requiring strategic thinking and quick reflexes.\n\nUltimately, the best virtual immersive game for you will depend on your preferences and interests. It\'s recommended to try out different games and see which one resonates with you the most.'